<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Insane_Membrane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##MemProtMD with Insane

[MemProtMD](https://doi.org/10.1016/j.str.2015.05.006) membrane set-up protocol for a preformed lipid membrane.

This uses the [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) represenation, with the lipid membrane constructed using [Insane](https://doi.org/10.1021/acs.jctc.5b00209) and run for 10 ns of molecular dynamics simulations using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

The final snapshot of the simulation is converted back to [CHARMM36m](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs) representation using [CG2AT](https://doi.org/10.1021/acs.jctc.1c00295); which can be downloaded from [here](https://github.com/pstansfeld/cg2at).

Inspiration for this Google Colab was taken from the amazing work of the [ColabFold](https://github.com/sokrypton/ColabFold) team.

Change the settings below and click **Runtime → Run all**.

In [ ]:
#@title Initialisation
%%capture
!pip install py3dmol
!apt-get update && apt-get upgrade -y && apt-get install -y gzip pymol dssp

#@markdown ---
MembraneType = "Other" #@param ["POPC 1","POPE 4 POPG 1","POPE 7 POPG 2 CARD 1","Asymmetric","Other"]
#@markdown #### Options for other membrane types:
if MembraneType == "Other":
  OtherMembraneComposition = "POPE 7 POPG 2 CARD 1" #@param {type:"string"}
  LowerMembraneComposition = OtherMembraneComposition
  UpperMembraneComposition = OtherMembraneComposition

#@markdown ---
#@markdown #### Options for asymmetric membranes:
elif MembraneType == "Asymmetric":
  UpperMembraneComposition = "KLA 1" #@param {type:"string"}
  LowerMembraneComposition = "POPE 7 POPG 2 CARD 1" #@param {type:"string"}
else:
  LowerMembraneComposition = MembraneType
  UpperMembraneComposition = MembraneType
#@markdown ---
#@markdown #### Define Periodic Unit:
  x = "12" #@param {type:"string"}
  y = "12" #@param {type:"string"}
  z = "12" #@param {type:"string"}
#@markdown ---
box = (float(x), float(y), float(z))

In [ ]:
#@title Configurations
import os
import sys
import requests
import py3Dmol
from google.colab import files

sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')

def py3dmol_view(pdbfilename, working_dir):
    mview = py3Dmol.view(width=800, height=400)
    with open(working_dir + pdbfilename, "r") as f:
        mol1 = f.read()
    mview.addModel(mol1, "pdb")
    mview.setStyle({"cartoon": {"color": "spectrum"}})
    mview.setStyle({'resn': 'DUM'}, {'sphere': {}})
    mview.setStyle({'atom': 'BB'}, {'sphere': {}})
    mview.setStyle({'atom': 'PO4'}, {'sphere': {}})
    mview.setStyle({'atom': 'P'}, {'sphere': {}})
    mview.setStyle({'atom': 'PO2'}, {'sphere': {}})
    mview.setStyle({'atom': 'PO1'}, {'sphere': {}})
    mview.setBackgroundColor("0xffffff")
    mview.zoomTo()
    mview.show()

def run_insane3(lipid, x, y, z, dm):
    lipid_args = lipid.split()
    cmd = [
        'python', '/content/insane3.py',
        *lipid_args, '-salt', '0.15', '-sol', 'W', '-o', 'CG-system.gro',
        '-p', 'topol.top', '-center',
        '-x', f"{x}", '-y', f"{y}", '-z', f"{z}"
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if stdout:
        print(stdout.decode('utf-8'))
    if stderr:
        print(stderr.decode('utf-8'))

def update_topology_file():
    replacements = {
        'NA+': 'NA',
        'CL-': 'CL',
        '#include "martini_v3.itp"': (
            '#include "martini_v3.0.0.itp"\n'
            '#include "martini_v3.0.0_ions_v1.itp"\n'
            '#include "martini_v3.0.0_solvents_v1.itp"\n'
            '#include "martini_v3.0.0_phospholipids_v1.itp"\n'
        ),
    }
    lines = []
    with open('topol.top') as infile:
        for line in infile:
            for src, target in replacements.items():
                line = line.replace(src, target)
            lines.append(line)
    with open('topol.top', 'w') as outfile:
        for line in lines:
            outfile.write(line)

def process_membrane_composition(composition, flag):
    lipid = ""

    input_list = composition.split()

    for i in range(0, len(input_list), 2):
        lipid_type = input_list[i]
        quantity = input_list[i + 1]
        lipid += f"{flag} {lipid_type}:{quantity} "
        if lipid_type == 'KLA':
            lipid += "-au 1.2 "

    lipid = lipid.strip()
    return lipid

upper_lipid = process_membrane_composition(UpperMembraneComposition, '-u')
lower_lipid = process_membrane_composition(LowerMembraneComposition, '-l')
lipid = upper_lipid + ' ' + lower_lipid

name = 'membrane'
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

In [ ]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/memembed/"):
  !pip install pdb2pqr vermouth gromacswrapper==0.8.3 MDAnalysis wget
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/BETA.pot
  !unzip -o martini_v300.zip
  !unzip -o gromacs.zip
  %mv /content/usr/local/gromacs/ /usr/local/
  !ln -s /usr/local/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil
import subprocess
import wget
import pandas as pd
from gromacs import grompp, editconf, make_ndx, trjconv, confrms, pdb2gmx, mdrun

In [ ]:
#@title Set-up Coarse-Grained Membrane System
#%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini_v300/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

run_insane3(lipid, round(box[0]), round(box[1]), round(box[2]), round(0))

update_topology_file()

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 1000\nemstep = 0.001')

grompp(f='em.mdp',o='em.tpr',c='CG-system.gro',maxwarn=10,backup=False,v=True)

mdrun(deffnm='em', c='CG-system.pdb',backup=False)

trjconv(f='CG-system.pdb', o='CG-system.pdb', pbc='res', s='em.tpr', conect=True, input='0',backup=False)

if not os.path.exists('MD'):
    os.mkdir('MD')

make_ndx(f='CG-system.pdb', o='index.ndx', input=('del 0-40', 'rW|rNA|rCL','!0','name 1 Lipid','name 0 SOL_ION','q'),backup=False)

with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = LIPID SOL_ION\ntau_t  = 1.0 1.0\nref_t  = 310 310\nPcoupl  = c-rescale\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')

grompp(f='cgmd.mdp',o='MD/md',c='CG-system.pdb',maxwarn=10, n='index.ndx',backup=False)

In [ ]:
#@title Equilibrate Membrane System
os.chdir(working_dir + 'MD')
mdrun(deffnm='md',backup=False, v=True, nsteps=10000, c='md.pdb')

py3dmol_view('md.pdb', f'{working_dir}MD/')

files_to_remove = [os.path.join(working_dir, f) for f in os.listdir(working_dir) if f.startswith("#")]
for file_path in files_to_remove:
    os.remove(file_path)


In [ ]:
#@title Convert back to Atomistic
os.chdir(working_dir)
shutil.rmtree('Atomistic', ignore_errors=True)

os.chdir(working_dir + 'MD/')
shutil.rmtree('CG2AT', ignore_errors=True)

# Convert CG to Atomistic
os.system('/content/cg2at/cg2at -group all -o align -w tip3p -c md.pdb -loc CG2AT -ff charmm36-jul2020-updated -fg martini_3-0_charmm36')

os.chdir(os.path.join(working_dir, 'MD/CG2AT/FINAL/'))

# Run GROMACS commands
grompp(f='../../../em.mdp', o='final.tpr', c='final_cg2at_de_novo.pdb', p='topol_final.top',maxwarn=10)

# Copy files
shutil.copytree(working_dir + 'MD/CG2AT/FINAL/', working_dir + 'Atomistic/')

os.chdir(os.path.join(working_dir, 'Atomistic'))
editconf(f='final.tpr',o=f"{name}-System.pdb")

# Rename files
os.rename(os.path.join(working_dir, 'MD', 'md.pdb'), os.path.join(working_dir, f"{name}-cgmd.pdb"))
os.rename(os.path.join(working_dir, 'MD', 'md.tpr'), os.path.join(working_dir, f"{name}-cgmd.tpr"))

# Create index file
os.replace(os.path.join(working_dir, 'Atomistic', 'topol_final.top'),
           os.path.join(working_dir, 'Atomistic', 'topol.top'))
make_ndx(f=working_dir + 'Atomistic/' + name + '-System.pdb', o='index.ndx', input=('del 0-40', 'rSOL|rNA*|rCL*','!0','name 0 water_and_ions','name 1 Lipid','q'),backup=False)

# Edit topology file
input_file = working_dir + 'Atomistic/topol.top'
temp_file = working_dir + 'Atomistic/temp_topol.top'

try:
    with open(input_file, 'r') as infile, open(temp_file, 'w') as outfile:
        filedata = infile.read()
        filedata = filedata.replace('../FINAL/', '')
        outfile.write(filedata)
    os.replace(temp_file, input_file)  # Rename temp file to original file
except Exception as e:
    print(f"An error occurred: {e}")
    if os.path.exists(temp_file):
        os.remove(temp_file)  # Clean up temp file if an error occurred

prod = 'https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/500ns-membrane.mdp'
filename = wget.download(prod)

In [ ]:
#@title View Atomistic System
py3dmol_view(f'Atomistic/{name}-System.pdb', working_dir)

In [ ]:
#@title Tidy up & Download
os.chdir(working_dir)
files_to_remove = [
    'em.trr',
    'em.tpr',
    'em.edr',
    'em.log',
    'mdout.mdp',
    'aligned.gro',
]

for file in files_to_remove:
    try:
        os.remove(file)
    except OSError:
        pass

try:
    shutil.rmtree('MD')
except OSError:
    pass

# create directory and move files
os.makedirs('CG', exist_ok=True)
for file_name in os.listdir('.'):
    if os.path.isfile(file_name):
        shutil.move(file_name, 'CG')

# zip and download
zip_name = name + '.zip'
os.chdir('/content/')
shutil.make_archive(name, 'zip', name)
files.download(zip_name)